In [ ]:
!pip install transformers
!pip install nltk
!pip install sentencepiece
!pip install pytorch-crf
!pip install scispacy==0.3.0
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz

     |████████████████████████████████| 2.2MB 11.5MB/s 
     |████████████████████████████████| 870kB 41.8MB/s 
     |████████████████████████████████| 3.3MB 36.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=1504054f502b8e4162cc11bb328686f52d926e12efc03802693dfae475d2cb86
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 10.5MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 10.4MB 12.0MB/s 
     |████████████████████████████████| 13.5MB 234kB/s 
     |████████████████████████████████| 1.1MB 37.6MB/s 
     |████████████████████████████████| 194kB 42.7MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers
from transformers import *
import spacy
import en_core_sci_sm
from google.colab import drive
import os
import re
from torchcrf import CRF

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='allenai/scibert_scivocab_uncased', vocab_size=31090, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
# nlp=spacy.load("en_core_web_sm")
nlp = en_core_sci_sm.load()

In [ ]:
def sent_tok(text):
  doc=nlp(text)
  return [s.text for s in doc.sents]

In [ ]:
drive.mount("/content/drive")
train_files=os.listdir("/content/drive/MyDrive/MeasEval/final_data/train/text")
val_files=os.listdir("/content/drive/MyDrive/MeasEval/final_data/dev/text")

Mounted at /content/drive


In [ ]:

ans = []
itr = 0
for fil in train_files:
  pathtxt=os.path.join("/content/drive/MyDrive/MeasEval/final_data/train/text",fil)
  pathtsv=os.path.join("/content/drive/MyDrive/MeasEval/final_data/train/tsv",fil[:-3]+"tsv")
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  tsv_lis = filetsv.values
  with open(pathtxt, 'r') as f:
    text_in=f.read()
  sen_lis = sent_tok(text=text_in)
  len_p = 0
  for sen in sen_lis:
    for i in range(len(tsv_lis)):
      if tsv_lis[i][2] == "Quantity" and tsv_lis[i][3]>=len_p and tsv_lis[i][4]<=len_p+len(sen):
        val = []
        val.append(sen)
        val.append((tsv_lis[i][3] - len_p, tsv_lis[i][4] - len_p))
        c = 0
        for j in range(len(tsv_lis)):
          if tsv_lis[j][2] == "MeasuredEntity" and tsv_lis[j][3]>=len_p and tsv_lis[j][4]<=len_p+len(sen) and tsv_lis[i][1] == tsv_lis[j][1]:
            c += 1
            val.append((tsv_lis[j][3] - len_p, tsv_lis[j][4] - len_p))
            ans.append(val)
        if c == 0:
          ans.append(val)
    len_p += len(sen)+1

In [ ]:
ans[2]

['Across the 5-year data cycles, 12.1% of participants without GHQ symptoms developed such symptoms, 4.6% of those without CES-D depressive symptoms had such symptoms at follow-up, and 2.1% of those not on antidepressant treatment started such medication.',
 (11, 17),
 (18, 29)]

In [ ]:

ans_val = []
for fil in val_files:
  pathtxt=os.path.join("/content/drive/MyDrive/MeasEval/final_data/dev/text",fil)
  pathtsv=os.path.join("/content/drive/MyDrive/MeasEval/final_data/dev/tsv",fil[:-3]+"tsv")
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  tsv_lis = filetsv.values
  with open(pathtxt, 'r') as f:
    text_in=f.read()
  sen_lis = sent_tok(text=text_in)
  len_p = 0
  for sen in sen_lis:
    for i in range(len(tsv_lis)):
      if tsv_lis[i][2] == "Quantity" and tsv_lis[i][3]>=len_p and tsv_lis[i][4]<=len_p+len(sen):
        val = []
        val.append(sen)
        val.append((tsv_lis[i][3] - len_p, tsv_lis[i][4] - len_p))
        c = 0
        for j in range(len(tsv_lis)):
          if tsv_lis[j][2] == "MeasuredEntity" and tsv_lis[j][3]>=len_p and tsv_lis[j][4]<=len_p+len(sen) and tsv_lis[i][1] == tsv_lis[j][1]:
            c += 1
            val.append((tsv_lis[j][3] - len_p, tsv_lis[j][4] - len_p))
            ans_val.append(val)
        if c == 0:
          ans_val.append(val)
    len_p += len(sen)+1

In [ ]:
ans_val[2]

['However, there is no detectable δ30Si offset between size fractions in the core tops TAN127 or 385 (Fig. 5a and b), whilst in KC08A (Fig. 5c), there is a slight negative offset of the 12–20 μm fraction which is opposite to that expected if the abundance of Chaetoceros was the cause (the suggested larger fractionation would act to lower δ30Si in the smaller size fractions).',
 (184, 192),
 (193, 201)]

In [ ]:
!pip install stanza

     |████████████████████████████████| 286kB 11.4MB/s 


In [ ]:
import stanza
stanza.download('en')
nlp2=stanza.Pipeline(lang='en', processors='tokenize', tokenize_no_ssplit=True)

2021-04-11 16:08:35 INFO: Downloading default packages for language: en (English)...
2021-04-11 16:10:00 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-04-11 16:10:00 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-04-11 16:10:00 INFO: Use device: gpu
2021-04-11 16:10:00 INFO: Loading: tokenize
2021-04-11 16:10:18 INFO: Done loading processors!


In [ ]:
def length(text):
  l = 0
  for i in range(len(text)):
    if text[i] != '#':
      l += 1
  return l

In [ ]:
pretrain = []
i = 0
for val in ans:
  label_q = []
  sen_tok = []
  sen = nlp2(val[0])
  label_en = []
  if len(val) == 3:
    for w in sen.sentences[0].words[:]:
      rng = re.findall(r'\d+', w.misc) 
      bert_tok = tokenizer.tokenize(w.text)
      sen_tok.extend(bert_tok)
      w_len = int(rng[0]);
      for t in bert_tok:
        if val[1][0] <= w_len and val[1][1] >= w_len + length(t):
          label_q.append(1)
        else:
          label_q.append(0)
        if val[2][0] <= w_len and val[2][1] >= w_len + length(t):
          label_en.append(1)
        else:
          label_en.append(0)
        w_len += length(t)
    for i in range(len(label_q)):
      if label_q[i] == 1 and i == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      elif label_q[i] == 1 and label_q[i-1] == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      if label_q[i] == 1 and i == len(label_q) - 1:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
      elif label_q[i] == 1 and label_q[i+1] == 0:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
    pretrain.append([sen_tok, label_en])
  else:
    for w in sen.sentences[0].words[:]:
      rng = re.findall(r'\d+', w.misc) 
      bert_tok = tokenizer.tokenize(w.text)
      sen_tok.extend(bert_tok)
      w_len = int(rng[0]);
      for t in bert_tok:
        if val[1][0] <= w_len and val[1][1] >= w_len + length(t):
          label_q.append(1)
        else:
          label_q.append(0)
        label_en.append(0)
        w_len += length(t)
    for i in range(len(label_q)):
      if label_q[i] == 1 and i == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      elif label_q[i] == 1 and label_q[i-1] == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      if label_q[i] == 1 and i == len(label_q) - 1:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
      elif label_q[i] == 1 and label_q[i+1] == 0:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
    pretrain.append([sen_tok, label_en])

In [ ]:
pretrain[2]

[['across',
  'the',
  '$',
  '5',
  '-',
  'year',
  '$',
  'data',
  'cycles',
  ',',
  '12',
  '.',
  '1',
  '%',
  'of',
  'participants',
  'without',
  'gh',
  '##q',
  'symptoms',
  'developed',
  'such',
  'symptoms',
  ',',
  '4',
  '.',
  '6',
  '%',
  'of',
  'those',
  'without',
  'ces',
  '-',
  'd',
  'depressive',
  'symptoms',
  'had',
  'such',
  'symptoms',
  'at',
  'follow',
  '-',
  'up',
  ',',
  'and',
  '2',
  '.',
  '1',
  '%',
  'of',
  'those',
  'not',
  'on',
  'antidepressant',
  'treatment',
  'started',
  'such',
  'medication',
  '.'],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [ ]:
validation = []
i = 0
for val in ans_val:
  label_q = []
  sen_tok = []
  sen = nlp2(val[0])
  label_en = []
  if len(val) == 3:
    for w in sen.sentences[0].words[:]:
      rng = re.findall(r'\d+', w.misc) 
      bert_tok = tokenizer.tokenize(w.text)
      sen_tok.extend(bert_tok)
      w_len = int(rng[0]);
      for t in bert_tok:
        if val[1][0] <= w_len and val[1][1] >= w_len + length(t):
          label_q.append(1)
        else:
          label_q.append(0)
        if val[2][0] <= w_len and val[2][1] >= w_len + length(t):
          label_en.append(1)
        else:
          label_en.append(0)
        w_len += length(t)
    for i in range(len(label_q)):
      if label_q[i] == 1 and i == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      elif label_q[i] == 1 and label_q[i-1] == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      if label_q[i] == 1 and i == len(label_q) - 1:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
      elif label_q[i] == 1 and label_q[i+1] == 0:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
    validation.append([sen_tok, label_en])
  else:
    for w in sen.sentences[0].words[:]:
      rng = re.findall(r'\d+', w.misc) 
      bert_tok = tokenizer.tokenize(w.text)
      sen_tok.extend(bert_tok)
      w_len = int(rng[0]);
      for t in bert_tok:
        if val[1][0] <= w_len and val[1][1] >= w_len + length(t):
          label_q.append(1)
        else:
          label_q.append(0)
        label_en.append(0)
        w_len += length(t)
    for i in range(len(label_q)):
      if label_q[i] == 1 and i == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      elif label_q[i] == 1 and label_q[i-1] == 0:
        label_en.insert(i, 0)
        sen_tok.insert(i,"$")
      if label_q[i] == 1 and i == len(label_q) - 1:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
      elif label_q[i] == 1 and label_q[i+1] == 0:
        label_en.insert(i+2, 0)
        sen_tok.insert(i+2,"$")
    validation.append([sen_tok, label_en])


In [ ]:
validation[2]

[['however',
  ',',
  'there',
  'is',
  'no',
  'detectable',
  'δ',
  '30',
  'si',
  'offset',
  'between',
  'size',
  'fractions',
  'in',
  'the',
  'core',
  'top',
  '##s',
  'tan',
  '##127',
  'or',
  '38',
  '##5',
  '(',
  'fig',
  '.',
  '5',
  'a',
  'and',
  'b',
  ')',
  ',',
  'whilst',
  'in',
  'kc',
  '##08',
  '##a',
  '(',
  'fig',
  '.',
  '5',
  'c',
  ')',
  ',',
  'there',
  'is',
  'a',
  'slight',
  'negative',
  'offset',
  'of',
  'the',
  '$',
  '12',
  '–',
  '20',
  'μ',
  '##m',
  '$',
  'fraction',
  'which',
  'is',
  'opposite',
  'to',
  'that',
  'expected',
  'if',
  'the',
  'abundance',
  'of',
  'cha',
  '##eto',
  '##cer',
  '##os',
  'was',
  'the',
  'cause',
  '(',
  'the',
  'suggested',
  'larger',
  'fractionation',
  'would',
  'act',
  'to',
  'lower',
  'δ',
  '30',
  'si',
  'in',
  'the',
  'smaller',
  'size',
  'fractions',
  ')',
  '.'],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [ ]:
train = []
for val in pretrain:
  lab = np.zeros(256)
  tok_arr = np.zeros(256)
  att_mask = np.zeros(256)
  sen_tok = []
  tok_arr[0] = 102
  att_mask[0] = 1
  for tok in val[0]:
    sen_tok.append(tokenizer.convert_tokens_to_ids(tok))
  for i in range(len(val[1])):
    lab[i+1] = val[1][i]
    tok_arr[i+1] = sen_tok[i]
    att_mask[i+1] = 1
  train.append([tok_arr,att_mask, lab])


In [ ]:
train[2]

[array([  102.,  2186.,   111.,  3250.,   305.,   579.,   996.,  3250.,
          453.,  5119.,   422.,   760.,   205.,   158.,  1863.,   131.,
         1914.,  1319.,  5357., 30147.,  3049.,  1815.,   555.,  3049.,
          422.,   286.,   205.,   370.,  1863.,   131.,  1052.,  1319.,
        10628.,   579.,   128., 11367.,  3049.,   883.,   555.,  3049.,
          235.,   589.,   579.,   692.,   422.,   137.,   170.,   205.,
          158.,  1863.,   131.,  1052.,   302.,   191., 21247.,   922.,
         6774.,   555.,  7297.,   205.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0., 

In [ ]:
val_data = []
for val in validation:
  lab = np.zeros(256)
  tok_arr = np.zeros(256)
  att_mask = np.zeros(256)
  sen_tok = []
  tok_arr[0] = 102
  att_mask[0] = 1
  for tok in val[0]:
    sen_tok.append(tokenizer.convert_tokens_to_ids(tok))
  for i in range(len(val[1])):
    lab[i+1] = val[1][i]
    tok_arr[i+1] = sen_tok[i]
    att_mask[i+1] = 1
  val_data.append([tok_arr,att_mask, lab])

In [ ]:
x_train_id = np.zeros((0,256))
x_train_mask = np.zeros((0,256))
y_train = np.zeros((0,256))
for val in train:
  x_train_id = np.vstack((x_train_id, val[0]))
  x_train_mask = np.vstack((x_train_mask, val[1]))
  y_train = np.vstack((y_train, val[2]))

In [ ]:
x_val_id = np.zeros((0,256))
x_val_mask = np.zeros((0,256))
y_val = np.zeros((0,256))
for val in val_data:
  x_val_id = np.vstack((x_val_id, val[0]))
  x_val_mask = np.vstack((x_val_mask, val[1]))
  y_val = np.vstack((y_val, val[2]))

In [ ]:
for i in range(len(y_train)):
  for j in range(1,len(y_train[0])):
    if y_train[i][j-1] == 0 and y_train[i][j] == 1:
      y_train[i][j] = 2
for i in range(len(y_val)):
  for j in range(1,len(y_val[0])):
    if y_val[i][j-1] == 0 and y_val[i][j] == 1:
      y_val[i][j] = 2

In [ ]:
from torch.utils.data import TensorDataset,DataLoader

train_data = TensorDataset(torch.from_numpy(x_train_id), torch.from_numpy(x_train_mask), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(x_val_id), torch.from_numpy(x_val_mask), torch.from_numpy(y_val))

In [ ]:
batch_size = 12
train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:

class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()
      self.bert = bert 
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*embed_dim,out_dim)
      self.w1 = nn.Linear(embed_dim, embed_dim)
      self.w2 = nn.Linear(embed_dim, embed_dim)
      #self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.softmax = nn.LogSoftmax(dim = 2)
      self.crf = CRF(3, batch_first=True)  
      self.tanh = nn.Tanh()

    #define the forward pass
    def forward(self, sent_id, mask_val, labels=None):
      x = self.bert(sent_id, attention_mask=mask_val)
      x = x.last_hidden_state
      x = self.tanh(x)
      cls = x[:,0,:]
      cls = cls.unsqueeze(1).repeat(1, 256, 1)
      cls = self.w1(cls)
      x = self.w2(x)
      x = torch.cat([x,cls], dim = 2)
      #x,_ = self.bilstm(x)
      x = self.dropout(x)
      x = self.fc1(x)
      mask_val = mask_val.type(torch.uint8)
      logit = self.softmax(x)
      if labels is not None:
          loss = -self.crf(logit, labels, mask=mask_val, reduction='mean')
          return loss
      else:
          prediction = self.crf.decode(x, mask=mask_val)
          return prediction

In [ ]:
bert_model = BERT_Arch(model, 768, 64, 0.1, 1,3)
bert_model = bert_model.to(device)

In [ ]:
print(bert_model)

BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [ ]:
optimizer = torch.optim.SGD(bert_model.parameters(), lr=0.01,momentum=0.9)

In [ ]:

def score(val_loader):
  p=0
  n=0
  pos=0
  neg=0
  prec_num = 0 
  prec_den = 0
  rec_num = 0
  rec_den = 0 
  p1 = 0
  n1=0
  for seq, mask, y in val_loader:
      bert_model.zero_grad()
      bert_model.eval()
      y_pred = bert_model(seq.long().to(device), mask.long().to(device))
      np_out = np.zeros((len(y_pred),256, 1))
      np_act = (y.cpu().data.numpy() >= 1).astype(int).reshape((len(y), len(y[0]),1))
      
      for i in range(np_out.shape[0]):
        prev_2 = 0
        for j in range(np_out.shape[1]):
          for k in range(np_out.shape[2]):
            if j >= len(y_pred[i]):
              np_out[i][j][k] = 0
            elif y_pred[i][j] == 2:
              np_out[i][j][k]=1
              prev_2 = 1
            elif y_pred[i][j] == 1 and prev_2 == 1:
              np_out[i][j][k]=1
            else:
              prev_2 = 0
              np_out[i][j][k]=0

      for i in range(np_out.shape[0]):
        for j in range(np_out.shape[1]):
          if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
            if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
              p1 = p1 + 1
            else:
              n1 = n1 + 1

      for i in range(np_out.shape[0]):
        for j in range(np_out.shape[1]):
          for k in range(np_out.shape[2]):
            if np_out[i,j,k] == np_act[i,j,k]:
              p = p +1
            else:
              n=n+1
            if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
              if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
                pos = pos + 1
              else:
                neg = neg + 1
            if np_out[i,j,k]==1:
              if np_out[i,j,k]==np_act[i,j,k]:
                prec_num = prec_num + 1
              else:
                prec_den = prec_den + 1
            if np_act[i,j,k]==1:
              if np_out[i,j,k]==np_act[i,j,k]:
                rec_num = rec_num + 1
              else:
                rec_den = rec_den + 1
  if prec_num+prec_den == 0:
    return 0
  precision = prec_num/(prec_num+prec_den)
  if rec_num+rec_den == 0:
    return 0
  recall = rec_num/(rec_num+rec_den)
  if precision == 0 and recall == 0:
    return 0
  F1 = 2*precision*recall/(precision+recall)
  return F1

In [ ]:
epochs = 10
for e in range(epochs):
  
  bert_model.train()
  i=0
  train_loss=0
  for seq, mask, y in train_loader:
    bert_model.zero_grad()
    loss = bert_model(seq.long().to(device), mask.long().to(device), y.long().to(device))
    train_loss += loss.item()*batch_size
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)
    optimizer.step()
    if(i%5==0):
      print("Epoch-{}/{} Iterations-{} loss-{}".format(e+1,epochs,i+1,loss.item()))
    i+=1
  
  bert_model.eval()
  f1_score = score(val_loader)
  val_loss=0
  for seq, mask, y in val_loader:
    bert_model.zero_grad()
    loss = bert_model(seq.long().to(device), mask.long().to(device), y.long().to(device))
    val_loss += loss.item()*batch_size
    i+=1
  
  print("Epoch-{}/{} train_loss-{} Val_loss-{} F1 Score-{}".format(e+1,epochs,train_loss/len(train_loader),val_loss/len(val_loader), f1_score))

Epoch-1/10 Iterations-1 loss-43.77806854248047
Epoch-1/10 Iterations-6 loss-17.45199203491211
Epoch-1/10 Iterations-11 loss-18.762617111206055
Epoch-1/10 Iterations-16 loss-17.09335708618164
Epoch-1/10 Iterations-21 loss-12.713094711303711
Epoch-1/10 Iterations-26 loss-9.867259979248047
Epoch-1/10 Iterations-31 loss-10.816783905029297
Epoch-1/10 Iterations-36 loss-11.462234497070312
Epoch-1/10 Iterations-41 loss-10.952098846435547
Epoch-1/10 Iterations-46 loss-8.620365142822266
Epoch-1/10 Iterations-51 loss-10.183650016784668
Epoch-1/10 Iterations-56 loss-5.192363739013672
Epoch-1/10 Iterations-61 loss-9.466171264648438
Epoch-1/10 Iterations-66 loss-8.430729866027832
Epoch-1/10 Iterations-71 loss-6.468984127044678
Epoch-1/10 Iterations-76 loss-11.56946086883545
Epoch-1/10 Iterations-81 loss-6.787550926208496
Epoch-1/10 Iterations-86 loss-7.947544097900391
Epoch-1/10 train_loss-146.35188484191895 Val_loss-97.49756455421448 F1 Score-0
Epoch-2/10 Iterations-1 loss-6.62139892578125
Epoch-2

In [ ]:
save_path = '/content/drive/MyDrive/MeasEval/scibert_crf_meas_ent_2iter.pt'
torch.save(bert_model, save_path)

In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in val_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.long().to(device), mask.long().to(device))
    np_out = np.zeros((len(y_pred),256, 1))
    np_act = (y.cpu().data.numpy() >= 1).astype(int).reshape((len(y), len(y[0]),1))
    
    for i in range(np_out.shape[0]):
      prev_2 = 0
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if j >= len(y_pred[i]):
            np_out[i][j][k] = 0
          elif y_pred[i][j] == 2:
            np_out[i][j][k]=1
            prev_2 = 1
          elif y_pred[i][j] == 1 and prev_2 == 1:
            np_out[i][j][k]=1
          else:
            prev_2 = 0
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------NER RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.35174418604651164
--------NER RESULTS--------
Accuracy:-0.9905315896739131
Modified Accuracy:-0.35174418604651164
Precision:-0.5707547169811321
Recall:-0.4782608695652174
F1 score:-0.5204301075268818
